In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
from tqdm import tqdm
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
import numpy as np
import shutil
import os

Using TensorFlow backend.


In [0]:
pwd

'/content'

In [0]:
cd ..

/


In [0]:
cd content/drive/My\ Drive/Projects_content/Game of Deep Learning CV Hackathon of AV

/content/drive/My Drive/Projects_content/Game of Deep Learning CV Hackathon of AV


In [0]:
output_dir = 'model_output/CNNwithImageDataGenerator'
seed = 7
np.random.seed(seed)
batch_size = 80

In [0]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test_ApKoW4T.csv')
train_df.head()


,image,category
0,2823080.jpg,1
1,2870024.jpg,1
2,2662125.jpg,2
3,2900420.jpg,3
4,2804883.jpg,2


In [0]:
test_df.shape,train_df.shape

((2680, 1), (6252, 2))

In [0]:
valid_df = train_df.iloc[int(train_df.shape[0]*(0.8)):,:]
# valid_df = train_df.iloc[0:2,:]

# valid_df.head()

train_df = train_df.iloc[:int(train_df.shape[0]*(0.8)),:]
valid_df.shape,train_df.shape

((1251, 2), (5001, 2))

In [0]:
valid_df = valid_df.reset_index(drop=True)
valid_df.head()

,image,category
0,2806710.jpg,1
1,2812752.jpg,5
2,2805770.jpg,2
3,2656405.jpg,4
4,2904583.jpg,5


In [0]:
dst_fldr_Cargo = r"images/train/Cargo";
dst_fldr_Military = r"images/train/Military";
dst_fldr_Carrier = r"images/train/Carrier";
dst_fldr_Cruise = r"images/train/Cruise";
dst_fldr_Tankers = r"images/train/Tankers";

dst_fldr_valid_Cargo = r"images/valid/Cargo";
dst_fldr_valid_Military = r"images/valid/Military";
dst_fldr_valid_Carrier = r"images/valid/Carrier";
dst_fldr_valid_Cruise = r"images/valid/Cruise";
dst_fldr_valid_Tankers = r"images/valid/Tankers";

try:
    os.makedirs(dst_fldr_Cargo); 
    os.makedirs(dst_fldr_Military);
    os.makedirs(dst_fldr_Carrier);
    os.makedirs(dst_fldr_Cruise);
    os.makedirs(dst_fldr_Tankers);
    os.makedirs(dst_fldr_valid_Cargo); 
    os.makedirs(dst_fldr_valid_Military);
    os.makedirs(dst_fldr_valid_Carrier);
    os.makedirs(dst_fldr_valid_Cruise);
    os.makedirs(dst_fldr_valid_Tankers);
except:
    print ("Folder already exist or some error")


Folder already exist or some error


In [0]:
datagen = ImageDataGenerator(width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True,rotation_range=40,zoom_range=0.2,vertical_flip=True)
train_it = datagen.flow_from_directory('images/train/',target_size=(250,250),classes=['Cargo','Carrier','Cruise','Military','Tankers'],batch_size=batch_size)
val_it = datagen.flow_from_directory('images/valid/',target_size=(250,250),classes=['Cargo','Carrier','Cruise','Military','Tankers'],batch_size=batch_size)
test_it = datagen.flow_from_directory('images/test/',target_size=(250,250),classes=['Cargo','Carrier','Cruise','Military','Tankers'],batch_size=batch_size)

Found 5001 images belonging to 5 classes.
Found 1251 images belonging to 5 classes.
Found 0 images belonging to 5 classes.


In [0]:
batchX, batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batch shape=(80, 250, 250, 3), min=0.000, max=255.000


In [0]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu", input_shape=(250,250,3)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.summary()

W0815 14:26:04.586457 140144899495808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0815 14:26:04.637646 140144899495808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0815 14:26:04.646906 140144899495808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0815 14:26:04.704660 140144899495808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0815 14:26:04.710825 140144899495808 deprecation_wrapp

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 248, 248, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 246, 246, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 123, 123, 64)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 123, 123, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 121, 121, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 119, 119, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 59, 59, 128)       0         
__________

In [0]:
# modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5",monitor='val_acc')
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.best.hdf5",monitor='val_acc',save_best_only=True, mode='max')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_it, steps_per_epoch=train_df.shape[0]//batch_size, validation_data=val_it, validation_steps=20,epochs=20,use_multiprocessing=True,shuffle=True,callbacks=[modelcheckpoint])

W0815 14:26:05.095323 140144899495808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0815 14:26:05.139169 140144899495808 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0815 14:26:05.291925 140144899495808 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
62/62 [==============================] - 2053s 33s/step - loss: 11.1296 - acc: 0.3077 - val_loss: 10.7112 - val_acc: 0.3355
Epoch 2/20
62/62 [==============================] - 144s 2s/step - loss: 10.9870 - acc: 0.3183 - val_loss: 11.2242 - val_acc: 0.3036
Epoch 3/20
62/62 [==============================] - 147s 2s/step - loss: 11.1271 - acc: 0.3097 - val_loss: 10.7317 - val_acc: 0.3342
Epoch 4/20
62/62 [==============================] - 144s 2s/step - loss: 10.9528 - acc: 0.3205 - val_loss: 10.8813 - val_acc: 0.3249
Epoch 5/20
62/62 [==============================] - 145s 2s/step - loss: 11.0876 - acc: 0.3121 - val_loss: 10.8138 - val_acc: 0.3291
Epoch 6/20
62/62 [==============================] - 145s 2s/step - loss: 11.1578 - acc: 0.3077 - val_loss: 10.8138 - val_acc: 0.3291
Epoch 7/20
62/62 [==============================] - 146s 2s/step - loss: 11.1637 - acc: 0.3074 - val_loss: 10.9882 - val_acc: 0.3183
Epoch 8/20
62/62 [==============================] - 146s 2s/step - 

In [0]:
model.load_weights(output_dir+"/weights.best.hdf5") # zero-indexed

In [0]:
pred = {}
def predictions(imagepath,imagename):
   
#   print(imagepath,imagename)
  img = image.load_img(imagepath,target_size=(250,250,3))
#   plt.imshow(img)
  img = image.img_to_array(img)
  proba = model.predict(img.reshape(1,250,250,3))   
  pred.update( {imagename : np.argmax(proba)+1} )  
#   print(imagename,np.argmax(proba)+1)
#   print(pred)
  
  

In [0]:
for id in range(valid_df.shape[0]):
  if (os.path.isfile(os.path.join(dst_fldr_valid_Cargo,valid_df['image'][id]))):    
      predictions(os.path.join(dst_fldr_valid_Cargo,valid_df['image'][id]),valid_df['image'][id])      
  elif (os.path.isfile(os.path.join(dst_fldr_valid_Military,valid_df['image'][id]))):
        predictions(os.path.join(dst_fldr_valid_Military,valid_df['image'][id]),valid_df['image'][id])
  elif (os.path.isfile(os.path.join(dst_fldr_valid_Carrier,valid_df['image'][id]))):
        predictions(os.path.join(dst_fldr_valid_Carrier,valid_df['image'][id]),valid_df['image'][id])
  elif (os.path.isfile(os.path.join(dst_fldr_valid_Cruise,valid_df['image'][id]))):
        predictions(os.path.join(dst_fldr_valid_Cruise,valid_df['image'][id]),valid_df['image'][id])
  elif (os.path.isfile(os.path.join(dst_fldr_valid_Tankers,valid_df['image'][id]))):
        predictions(os.path.join(dst_fldr_valid_Tankers,valid_df['image'][id]),valid_df['image'][id])                
        
      

In [0]:
pred_df = pd.DataFrame(list(pred.items()), columns=['Image', 'prediction'])
pred_df.head()
y_hat = pred_df['prediction']
y_valid = valid_df['category']

In [0]:
_val_f1 = f1_score(y_valid, y_hat,average='micro')
print(_val_f1)

0.32454036770583533


In [0]:
# model = Sequential()
# model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(300,300,3)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# # model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# # model.add(Dropout(0.25))
# # model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# # model.add(Dropout(0.25))
# # model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# # model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(6, activation='softmax'))

In [0]:
# model.summary()

In [0]:
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
# model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)